# Intelligent Query Processing
## Batch Mode on Rowstore
Batch mode on Rowstore is an example of Intelligent Query Processing that can be enabled by using dbcompat = 150. Follow the steps in this notebook to see the performance difference when using batch mode on rowstore.

**Prerequisites**: Restore the WideWorldImportersDW full backup. Then run the **extendwwidw.sql** script to customize the database. If you have already done this, proceed to the next steps

## Step 1: Do some setup
Warm the buffer pool cache to make it a fair comparison.

In [6]:
-- Step 1: Warm the buffer pool cache to make it a fair comparison
USE [WideWorldImportersDW]
GO
SELECT COUNT(*) FROM Fact.OrderHistoryExtended
GO

Commands completed successfully.

Total execution time: 00:00:00.0014800

(1 row affected)

Total execution time: 00:00:00.4034661

(No column name)
29620736


## Step 2: Prepare the query using dbcompat = 130
Clear the procedure cache and change dbcompat to 130 to ensure batch mode for rowstore is not enabled

In [7]:
-- Step 2: Clear the procedure cache and change dbcompat to 130 to ensure batch mode for rowstore is not enabled
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE
GO
ALTER DATABASE WideWorldImportersDW SET COMPATIBILITY_LEVEL = 130
GO

Commands completed successfully.

Total execution time: 00:00:00.0053028

Commands completed successfully.

Total execution time: 00:00:00.0091220

## Step 3: Run a typical data analytics query that uses aggregration and ordering
Run a query that gets tax rates, order lineage keys, order salesperson, and the sum of orders by salesperson. Study the results of a profile of the query including what operators and total rows per operator were used.

In [3]:
-- Step 3: Run the query
SET STATISTICS PROFILE ON
GO
SELECT [Tax Rate], [Lineage Key], [Salesperson Key], SUM(Quantity) AS SUM_QTY, 
SUM([Unit Price]) AS SUM_BASE_PRICE, COUNT(*) AS COUNT_ORDER
FROM Fact.OrderHistoryExtended
WHERE [Order Date Key]<=DATEADD(dd, -73, '2015-11-13')
GROUP BY [Tax Rate], [Lineage Key], [Salesperson Key]
ORDER BY [Tax Rate], [Lineage Key], [Salesperson Key]
GO
SET STATISTICS PROFILE OFF
GO

Commands completed successfully.

Total execution time: 00:00:00.0011701

(100 rows affected)

(8 rows affected)

Total execution time: 00:00:06.1683693

Commands completed successfully.

Total execution time: 00:00:00.0015284

Tax Rate,Lineage Key,Salesperson Key,SUM_QTY,SUM_BASE_PRICE,COUNT_ORDER
15.000,9,4,433280,243367.68,9216
15.000,9,6,836480,543988.48,18432
15.000,9,7,724480,846118.40,24576
15.000,9,8,862080,1028780.80,22144
15.000,9,9,1101056,924917.76,28928
15.000,9,11,2279296,3886913.28,62848
15.000,9,12,3262592,3852878.08,85888
15.000,9,15,5484800,6343098.88,144256
15.000,9,19,91831040,107590420.48,2310528
15.000,9,21,1240960,2060285.44,37632


Rows,Executes,StmtText,StmtId,NodeId,Parent,PhysicalOp,LogicalOp,Argument,DefinedValues,EstimateRows,EstimateIO,EstimateCPU,AvgRowSize,TotalSubtreeCost,OutputList,Warnings,Type,Parallel,EstimateExecutions
100,1,"SELECT [Tax Rate], [Lineage Key], [Salesperson Key], SUM(Quantity) AS SUM_QTY, SUM([Unit Price]) AS SUM_BASE_PRICE, COUNT(*) AS COUNT_ORDER FROM Fact.OrderHistoryExtended WHERE [Order Date Key]<=DATEADD(dd, -73, '2015-11-13') GROUP BY [Tax Rate], [Lineage Key], [Salesperson Key] ORDER BY [Tax Rate], [Lineage Key], [Salesperson Key]",1,1,0,NULL,NULL,NULL,NULL,202,NULL,NULL,NULL,611.7424,NULL,NULL,SELECT,0,NULL
0,0,"|--Compute Scalar(DEFINE:([Expr1005]=CONVERT_IMPLICIT(int,[globalagg1009],0)))",1,3,1,Compute Scalar,Compute Scalar,"DEFINE:([Expr1005]=CONVERT_IMPLICIT(int,[globalagg1009],0))","[Expr1005]=CONVERT_IMPLICIT(int,[globalagg1009],0)",202,0,2.02E-05,49,611.7424,"[WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [Expr1003], [Expr1004], [Expr1005]",NULL,PLAN_ROW,0,1
100,1,"|--Parallelism(Gather Streams, ORDER BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] ASC))",1,4,3,Parallelism,Gather Streams,"ORDER BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] ASC)",NULL,202,0,0.02987292,53,611.7424,"[WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [Expr1003], [Expr1004], [globalagg1009]",NULL,PLAN_ROW,1,1
100,8,"|--Stream Aggregate(GROUP BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key]) DEFINE:([Expr1003]=SUM([partialagg1006]), [Expr1004]=SUM([partialagg1007]), [globalagg1009]=SUM([partialagg1008])))",1,5,4,Stream Aggregate,Aggregate,"GROUP BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key])","[Expr1003]=SUM([partialagg1006]), [Expr1004]=SUM([partialagg1007]), [globalagg1009]=SUM([partialagg1008])",202,0,0.00014645,53,611.7125,"[WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [Expr1003], [Expr1004], [globalagg1009]",NULL,PLAN_ROW,1,1
800,8,"|--Sort(ORDER BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] ASC))",1,6,5,Sort,Sort,"ORDER BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] ASC)",NULL,808,0.002815315,0.003068509,53,611.7123,"[WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [partialagg1006], [partialagg1007], [partialagg1008]",NULL,PLAN_ROW,1,1
800,8,"|--Parallelism(Repartition Streams, Hash Partitioning, PARTITION COLUMNS:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[L

## Step 4: Study the query plan
Take the query above and paste it into a new query window using the WideWorldImportersDW database. Then hit `<ctrl`>+`<shift`>+`<p`>. Type the word **Run**. Select Run Current Query with Actual Plan. You should get a window that now shows you the actual graphic plan and it will look like this![query_plan_without_batchmode_on_rowstore](/sqlbk/ch2_intelligent_performance/iqp/batchmoderow/query_using_rowmode.png)

If you hover over the Table scan operator you can see the **Actual Execution Mode** = **Row**. You will also see that there is a Paralleism operator for the Hash Match and that operator has to process around 800 rows.

**TODO**: June build of ADS should allow this to be a local relative path. Fix before book publish

## Step 5: Prepare the query with dbcompat = 150 to enable Batch Mode on Rowstore

In [8]:
-- Step 4: Clear the procedure cache and change dbcompat to 150 to enable batch mode
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE
GO
ALTER DATABASE WideWorldImportersDW SET COMPATIBILITY_LEVEL = 150
GO

Commands completed successfully.

Total execution time: 00:00:00.0044057

Commands completed successfully.

Total execution time: 00:00:00.0083674

## Step 6: Run the query again to see the new performance and profile

In [5]:
-- Step 5: Now let's run it again and see if it is much faster
SET STATISTICS PROFILE ON
GO
SELECT [Tax Rate], [Lineage Key], [Salesperson Key], SUM(Quantity) AS SUM_QTY, 
SUM([Unit Price]) AS SUM_BASE_PRICE, COUNT(*) AS COUNT_ORDER
FROM Fact.OrderHistoryExtended
WHERE [Order Date Key]<=DATEADD(dd, -73, '2015-11-13')
GROUP BY [Tax Rate], [Lineage Key], [Salesperson Key]
ORDER BY [Tax Rate], [Lineage Key], [Salesperson Key]
GO
SET STATISTICS PROFILE OFF
GO

Commands completed successfully.

Total execution time: 00:00:00.0022125

(100 rows affected)

(6 rows affected)

Total execution time: 00:00:01.8824296

Commands completed successfully.

Total execution time: 00:00:00.0015083

Tax Rate,Lineage Key,Salesperson Key,SUM_QTY,SUM_BASE_PRICE,COUNT_ORDER
15.000,9,4,433280,243367.68,9216
15.000,9,6,836480,543988.48,18432
15.000,9,7,724480,846118.40,24576
15.000,9,8,862080,1028780.80,22144
15.000,9,9,1101056,924917.76,28928
15.000,9,11,2279296,3886913.28,62848
15.000,9,12,3262592,3852878.08,85888
15.000,9,15,5484800,6343098.88,144256
15.000,9,19,91831040,107590420.48,2310528
15.000,9,21,1240960,2060285.44,37632


Rows,Executes,StmtText,StmtId,NodeId,Parent,PhysicalOp,LogicalOp,Argument,DefinedValues,EstimateRows,EstimateIO,EstimateCPU,AvgRowSize,TotalSubtreeCost,OutputList,Warnings,Type,Parallel,EstimateExecutions
100,1,"SELECT [Tax Rate], [Lineage Key], [Salesperson Key], SUM(Quantity) AS SUM_QTY, SUM([Unit Price]) AS SUM_BASE_PRICE, COUNT(*) AS COUNT_ORDER FROM Fact.OrderHistoryExtended WHERE [Order Date Key]<=DATEADD(dd, -73, '2015-11-13') GROUP BY [Tax Rate], [Lineage Key], [Salesperson Key] ORDER BY [Tax Rate], [Lineage Key], [Salesperson Key]",1,1,0,NULL,NULL,NULL,NULL,202,NULL,NULL,NULL,575.3655,NULL,NULL,SELECT,0,NULL
100,1,"|--Parallelism(Gather Streams, ORDER BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] ASC))",1,2,1,Parallelism,Gather Streams,"ORDER BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] ASC)",NULL,202,0,0.02985898,49,575.3655,"[WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [Expr1003], [Expr1004], [Expr1005]",NULL,PLAN_ROW,1,1
100,8,"|--Sort(ORDER BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] ASC))",1,3,2,Sort,Sort,"ORDER BY:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] ASC, [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] ASC)",NULL,202,0.0009384385,8.534364E-05,49,575.3357,"[WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [Expr1003], [Expr1004], [Expr1005]",NULL,PLAN_ROW,1,1
100,8,"|--Compute Scalar(DEFINE:([Expr1005]=CONVERT_IMPLICIT(int,[Expr1011],0)))",1,4,3,Compute Scalar,Compute Scalar,"DEFINE:([Expr1005]=CONVERT_IMPLICIT(int,[Expr1011],0))","[Expr1005]=CONVERT_IMPLICIT(int,[Expr1011],0)",202,0,0,49,575.3347,"[WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [Expr1003], [Expr1004], [Expr1005]",NULL,PLAN_ROW,1,1
100,8,"|--Hash Match(Aggregate, HASH:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key]), RESIDUAL:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] = [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] AND [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] = [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key] AND [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key] = [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key]) DEFINE:([Expr1003]=SUM([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Quantity]), [Expr1004]=SUM([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Unit Price]), [Expr1011]=COUNT(*)))",1,5,4,Hash Match,Aggregate,"HASH:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Lineage Key], [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Salesperson Key]), RESIDUAL:([WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] = [WideWorldImportersDW].[Fact].[OrderHistoryExtended].[Tax Rate] AND [

## Step 7: Compare the new query plan
The query with batch mode ran about 2x faster and the resulting query plan can explain why. Take the query above and paste it into a new query window using the WideWorldImportersDW database. Then hit `<ctrl`>+`<shift`>+`<p`>. Type the word **Run**. Select Run Current Query with Actual Plan. You should get a window that now shows you the actual graphic plan and it will look like this![query_plan_with_batchmode_on_rowstore](/sqlbk/ch2_intelligent_performance/iqp/batchmoderow/query_using_batchmode.png)

If you hover over the Table scan operator you can see the **Actual Execution Mode** = **Batch**. You will also see that there is only one Paralleism operator and the Hash Match operator only proceses around 100 rows.

**TODO**: June build of ADS should allow this to be a local relative path. Fix before book publish